# «Сведения о месте осуществления трудовой деятельности застрахованного лица»

In [1]:
from datetime import datetime
now = datetime.now()
print('Время проверки', now)

Время проверки 2020-06-26 17:54:11.113788


### Итог тестирования

Лог файл:
[InfoWorkZL.csv](InfoWorkZL.csv)

In [9]:
print(test_result_)

Тест пройден без ошибок


---

### Методика тестирования

1. Отбираем 10 записей для проверки [sql.sql](sql.sql)
2. Результат на САИП [result.csv](result.csv)

In [2]:
import pandas as pd
from lxml import objectify

In [3]:
result_sql = pd.read_csv('result.csv')
result_sql.head(10)

,snils,lastname,firstname,middlename,min_period,max_period
0,100598777,1612964779,810656256,-1920524319,10.2016,5.2019
1,100615549,1663819752,-940686630,-1920524319,10.2016,5.2019
2,100615852,1853489284,-940686630,1312689740,10.2016,5.2019
3,100627960,1188053875,1569876569,1062608582,10.2016,5.2019
4,100641550,1337842800,2117846202,2088331112,10.2016,7.2017
5,100648564,-918524267,-658450477,1731495413,4.2016,4.2016
6,100653254,-545226514,1354076042,-527269213,10.2016,5.2019
7,100660049,1276475267,1201627000,-644063894,10.2016,5.2019
8,101055138,-63766679,1569876569,-664682458,4.2016,4.2016
9,101071641,398018782,-578808076,-715357757,10.2016,5.2019


### Проверяем сервис по отобранным снилсам

Если xml выдает поля в соответствии с выборкой из БД, считаем тест пройден

In [4]:
f = open('result.csv', 'r', encoding='utf-8')
f.readline()
content = f.readlines()
list_of_result = {}
base_url = 'http://10.138.243.194:8080/report/query?query=InfoWorkZL'
for line in content:
    line = line.strip().split(',')
    snils = str(line[0]).replace('"', '')
    lastname = str(line[1]).replace('"', '')
    firstname = str(line[2]).replace('"', '')
    patronymic = str(line[3]).replace('"', '')
    period_from = str(line[4]).replace('"', '')
    if len(str(period_from)) < 7:
        period_from = '0' + str(period_from)
    period_to = str(line[5]).replace('"', '')
    if len(str(period_to)) < 7:
        period_to = '0' + str(period_to)
    url = base_url + '&snils=' + snils + '&lastname=' + lastname + '&firstname=' + firstname + '&patronymic=' + patronymic + '&period_from=' + period_from + '&period_to=' + period_to
    try:
        xml = objectify.parse(url).getroot()
        if str(xml.SNILS) == snils and str(xml.Lastname) == lastname and str(xml.Firstname) == firstname and str(xml.Patronymic) == patronymic:
            test_result = 'OK', url
            list_of_result[snils] = test_result
        else:
            test_result = 'Error', url
            list_of_result[snils] = test_result
    except:
        test_result = 'Error', 'Какая то вторая странная ошибка', url
        list_of_result[snils] = test_result  

In [6]:
## Ошибки:
i = 0
for Error in list_of_result.values():
    if 'Error' in Error:
        i += 1
        print(Error)
if i == 0:
    test_result_ = 'Тест пройден без ошибок'
else:
    test_result_ = 'Количество ошибок:' +  i

In [10]:
# Также визуально проверяем xml по одному снилсу:
for appt in xml.getchildren():
    print("%s => %s" % (appt.tag, appt.text))
    for e in appt.getchildren():
        print("%s => %s" % (e.tag, e.text))
        for a in e.getchildren():
            print("%s => %s" % (a.tag, a.text))
    print()

{http://kvs.pfr.com/work-fact/1.0.1}Lastname => 398018782

{http://kvs.pfr.com/work-fact/1.0.1}Firstname => -578808076

{http://kvs.pfr.com/work-fact/1.0.1}Patronymic => -715357757

{http://kvs.pfr.com/work-fact/1.0.1}SNILS => 101071641

{http://kvs.pfr.com/work-fact/1.0.1}Period_from => 10.2016

{http://kvs.pfr.com/work-fact/1.0.1}Period_to => 05.2019

{http://kvs.pfr.com/work-fact/1.0.1}InfoWork => None
{http://kvs.pfr.com/work-fact/1.0.1}Year => 2016
{http://kvs.pfr.com/work-fact/1.0.1}Month => 10
{http://kvs.pfr.com/work-fact/1.0.1}Insurant => None
{http://kvs.pfr.com/work-fact/1.0.1}Insurant_INN => 525406658558
{http://kvs.pfr.com/work-fact/1.0.1}Insurant_KPP => None
{http://kvs.pfr.com/work-fact/1.0.1}Insurant_Name => None
{http://kvs.pfr.com/work-fact/1.0.1}Insurant_Address => 607186 - - - - ШЕВЧЕНКО УЛ 29 - 17

{http://kvs.pfr.com/work-fact/1.0.1}InfoWork => None
{http://kvs.pfr.com/work-fact/1.0.1}Year => 2016
{http://kvs.pfr.com/work-fact/1.0.1}Month => 11
{http://kvs.pfr.com

In [12]:
## Запись лога в файл:
with open('InfoWorkZL.csv', 'w') as f3write:
    for key,val in list_of_result.items():
        f3write.write('{}:{}\n'.format(key,val))